In [ ]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Overview - Image Augmentation for AutoML Vision

### Topic

This notebook demonstrates adding image augmentation using the AutoML API for Vision Classification.

### Audience

The audience for this notebook are software engineers (SWE) with limited experience in machine learning (ML) and data scientists seeking to add automatic image augmentation when using AutoML Vision Classification.

### Prerequistes

One should be familar with:

    - Python 3.X
    - Google Cloud Platform (GCP) and using GCP buckets.
    - Concept of Image Classification.
    - Concept of Image Augmentation.
    
### Dataset

This notebook using the builtin dataset for flowers stored on a Google public GCS bucket, located at:

    gs://cloud-ml-data/img/flower_photos/
    
This dataset contains train and test images for training an image classifier to recognize the following flower types:

    - Daisy
    - Dandelion
    - Roses
    - Sunflowers
    - Tulips

### Objective

The objective of this tutorial is to learn how to use the AutoML API to add image augmentation to an existing dataset.

### Costs 

This tutorial uses billable components of AutoML Vision.

Learn about [AutoML Vision Pricing](https://cloud.google.com/vision/automl/pricing)

### Set up your local development environment

**If you are using Colab or AI Platform Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* The Google AutoML SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3
* OpenCV

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

2. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

3. [Install AutoML SDK](https://pypi.org/project/google-cloud-automl/) using the `pip install google-cloud-automl` command in a shell.

4. [Install OpenCV](https://pypi.org/project/opencv-python/) using the `pip install opencv-python`

5. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3.

6. Activate that environment and run `pip install jupyter` in a shell to install
   Jupyter.

7. Run `jupyter notebook` in a shell to launch Jupyter.

8. Open this notebook in the Jupyter Notebook Dashboard.

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager)

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

Jupyter runs lines prefixed with `%` as automagic commands, which are interpreted within your IPython session. Automagic commands include `%ls`, `%pwd`, `%env` and `%pip` for example.

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}
!gcloud config set project $PROJECT_ID

### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the GCP Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. From the **Service account** drop-down list, select **New service account**.

3. In the **Service account name** field, enter a name.

4. From the **Role** drop-down list, select
   **Machine Learning Engine > AI Platform Admin** and
   **Storage > Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

### Set Service Account Role for AutoML

Add yourself and your service account to the AutoML Editor IAM role.

    - Replace your-userid@your-domain with your user account.
    - Replace service-account-name with the name of your new service account, for example service-account1@myproject.iam.gserviceaccount.com.

In [ ]:
!gcloud auth login
!gcloud projects add-iam-policy-binding $PROJECT_ID \
   --member="[user:your-userid@your-domain]" \
   --role="roles/automl.admin"
!gcloud projects add-iam-policy-binding $PROJECT_ID \
   --member="[serviceAccount:service-account-name]" \
   --role="roles/automl.editor"

### Allow AutoML service to access for Google Cloud project

Allow the AutoML service account (custom-vision@appspot.gserviceaccount.com) to access your Google Cloud project resources:

*This is a pre-existing global AutoML Vision service account which is separate from the project service account you just created; it is not visible in your project's list of service accounts.*

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:custom-vision@appspot.gserviceaccount.com" \
  --role="roles/ml.admin"
!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:custom-vision@appspot.gserviceaccount.com" \
  --role="roles/storage.admin"
!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:custom-vision@appspot.gserviceaccount.com" \
  --role="roles/serviceusage.serviceUsageAdmin"

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. You can then
create an AI Platform model version based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

You may also change the `COMPUTE_REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

In [ ]:
BUCKET_NAME = PROJECT_ID + "-vcm"        #@param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
# Default compute region for AutoML
COMPUTE_REGION='us-central1'

! gsutil mb -l $COMPUTE_REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### PIP Install Packages and dependencies

Install addional dependencies not install in Notebook environment
(e.g. XGBoost, adanet, tf-hub)

- Use the latest major GA version of the framework.

In [ ]:
!pip install -U google-cloud-storage
!pip install -U opencv-python

# Tutorial

### Import libraries and define constants

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

# import the Google AutoML client library
from google.cloud import automl_v1beta1 as automl

### Copy the sample images locally

Copy the flower dataset used in this [Tensorflow blog](https://cloud.google.com/blog/products/gcp/how-to-classify-images-with-tensorflow-using-google-cloud-machine-learning-and-cloud-dataflow) post. The images are stored in a public Cloud Storage bucket, so you can copy them directly from there.

This may take a couple of minutes to complete.

In [ ]:
!gsutil -m cp -R gs://cloud-ml-data/img/flower_photos/ ./

### Image Augmentation

Now that we have all the images in a bucket, we will scan through each folder (e.g., daisy) and augment each image (make a new augmented image) in the folder, as follows:

    - Vertical Flip (mirror)
    - Rotate 90 degrees
    - Center Crop to 90%
    
The new augmented images will then be added back to the corresponding folder. When all done, each folder will have 4X the number of original images.

In [ ]:
import os

def crop_center(image,cropx,cropy):
    """ Center Crop an image
    """
    H, W, C = image.shape
    startx = W//2-(cropx//2)
    starty = H//2-(cropy//2)    
    return image[starty:starty+cropy,startx:startx+cropx]

def augment_subdir(subdir):
    """ Augment all the images in a sub-folder
        - Vertical Flip (mirror)
        - Rotate 90 degrees
        - Center Crop to 90% of original
    """

    # Get list of the images in the bucket
    files=!ls $subdir
    for file in files:
        image = cv2.imread(subdir + "/" + file)

        try:
            # vertical flip
            mirror = np.flip(image, 0)
            # upload augmented image back to GCS bucket
            new_file = subdir + "/m_" + file
            cv2.imwrite(new_file, mirror)
            print(new_file)
        except: pass
        
        try:
            # rotate 90m degrees
            rot    = np.rot90(image, 1)
            # upload augmented image back to GCS bucket
            new_file = subdir + "/r_" + file
            cv2.imwrite(new_file, rot)
        except: pass

        try:
            # Center Crop the image by 10%
            H = int(image.shape[0] * 0.9)
            W = int(image.shape[1] * 0.9)
            crop = crop_center(image, W, H)
            # upload augmented image back to GCS bucket
            new_file = subdir + "/c_" + file
            cv2.imwrite(new_file, crop)
        except: pass
        print(subdir, file)
        
augment_subdir("flower_photos/daisy")
augment_subdir("flower_photos/dandelion")
augment_subdir("flower_photos/roses")
augment_subdir("flower_photos/sunflowers")
augment_subdir("flower_photos/tulips")

### Copy the sample+augmented images into your bucket

Copy the augmented flower dataset to your own bucket.

This may take a couple of minutes to complete.

In [ ]:
!gsutil -m cp -R flower_photos/ gs://$BUCKET_NAME/img/

### Create the CSV file

The sample dataset contains a CSV file with all of the image locations and the labels for each image. You'll use that to create your own CSV file:

    1. Update the CSV file to point to the files in your own bucket.
    2. Copy the CSV file into your bucket.
    
To learn more about preparing your training data, see [Preparing Training Data](https://cloud.google.com/vision/automl/docs/prepare)

In [ ]:
import os
csv_file = open("all_data.csv", "w")
subdirs = os.scandir('flower_photos')
for subdir in subdirs:
    if subdir.is_dir():
        files = os.scandir(subdir.path)
        for file in files:
            suffix = file.name.split('.')[-1]
            if suffix in ['jpg', 'jpeg', 'png', 'gif', 'bmp']:
                csv_file.write('gs://' + BUCKET_NAME + '/img/' + file.path + "," + subdir.name + "\n")
csv_file.close()

!gsutil cp all_data.csv gs://$BUCKET_NAME/csv/

### Create and Configure an AutoML client instance

In [ ]:
# Create an AutoML client
client = automl.AutoMlClient()

# Derive the full GCP path to the project
project_location = client.location_path(PROJECT_ID, COMPUTE_REGION)

### Creating a dataset

A dataset contains representative samples of the type of content you want to classify, labeled with the category labels you want your custom model to use. The dataset serves as the input for training a model.

The main steps for building a dataset are:

    - Specify a name for the dataset.
    - Create a dataset and specify whether to allow multiple labels on each item.
    - Import data items into the dataset.
    
The first step in creating a custom model is to create an empty dataset that will eventually hold the training data for the model. When you create a dataset, you specify the type of classification you want your custom model to perform:

    MULTICLASS assigns a single label to each classified document
    MULTILABEL allows a document to be assigned multiple labels
    
*In this tutorial, we are doing a MULTICLASS dataset*

In [ ]:
# Specify a name for the dataset
DATASET_NAME="[my-dataset-name]"

# Specify the image classification type for the dataset.
dataset_metadata = {"classification_type": 'MULTICLASS'}
# Set dataset name and metadata of the dataset.
my_dataset = {
    "display_name": DATASET_NAME,
    "image_classification_dataset_metadata": dataset_metadata,
}

# Create a dataset with the dataset metadata in the region.
response = client.create_dataset(project_location, my_dataset)

Display response for creating an empty dataset.

In [ ]:
# Display the dataset information.
print("Dataset name: {}".format(response.name))
print("Dataset id: {}".format(response.name.split("/")[-1]))
print("Dataset display name: {}".format(response.display_name))
print("Image classification dataset metadata:")
print("\t{}".format(response.image_classification_dataset_metadata))
print("Dataset example count: {}".format(response.example_count))

# Save the dataset ID
dataset_id = response.name.split("/")[-1]

### Importing items into a dataset

After you have created a dataset, you can import item URIs and labels for items from a CSV file stored in a Google Cloud Storage bucket. 

In [ ]:
# Get the full path of the dataset.
dataset_full_id = client.dataset_path(
    PROJECT_ID, COMPUTE_REGION, dataset_id
)

# Specify the location of the CSV file for the dataset
CSV_DATASET = "gs://" + BUCKET_NAME + "/csv/all_data.csv"
input_config = {"gcs_source": {"input_uris": [CSV_DATASET]}}

# Import data from the input URI.
response = client.import_data(dataset_full_id, input_config)


Display response from initiating the import of images into the dataset. **Call will return when import has completed. This may take upto 20 minutes**

In [ ]:
# synchronous check of operation status.
print("Data imported. {}".format(response.result()))

### Listing Datasets

A project can have multiple datasets, each used to train a separate model. You can get a list of the available datasets and can delete datasets you no longer need.

In [ ]:
response = client.list_datasets(project_location, None)

print("List of datasets:")
for dataset in response:
    # Display the dataset information.
    print("Dataset name: {}".format(dataset.name))
    print("Dataset id: {}".format(dataset.name.split("/")[-1]))
    print("Dataset display name: {}".format(dataset.display_name))
    print("Image classification dataset metadata:")
    print("\t{}".format(dataset.image_classification_dataset_metadata))
    print("Dataset example count: {}\n".format(dataset.example_count))